<h3>I was interested in seeing where the largest occurances of coronavirus cases had occurred in the UK in terms of individual counties.</h3>

I have combined two datasets for this analysis:

    (i) scratching data the governmnet website (22nd May 2020).
    (ii) area polygons for each council - this was a json file  downloaded from a government website.

For each local authority area a grey colour scale is given based on the number of confirmed cases. Also each council has a pop-up box containing information on that council.

The output is an interactive map viewed in a web browser.

    Mike Bourne

In [2]:
import folium 
import pandas
import json
from colour import Color




# definition to return colour code based on input number
# in this case number is number of covid cases in an English county upto 22nd June

def col_GOR(number):
    if(number < 1000):
        return 'green'
    elif ( 1000 < number < 2500):
        return 'orange'
    else:
        return 'red'

# This function takes a number and assigns a grey colour for it
# based on a min to max range. 
def col_grey(number, min_no=0, max_no=5000):
    if number > max_no : max_no = number +1
    number = number / max_no * 100 
    colours = list(Color("white").range_to(Color("black"),100))

    if(int(number)-1 <= 0): return 'white'
    else:
        return str(colours[int(number)-1])
    
# read in csv file containing the covid-19 data for England data. 
# create dictionary from this csv; number of cases (cases) and couty names (names)

covid_eng_df = pandas.read_csv('covid_eng.csv', usecols=['TotalCases','AreaCode'])

covid_numbers = covid_eng_df.set_index('AreaCode')['TotalCases'].to_dict()


# read in json file containing the other data for England coutnies (eg lat, long).

with open('Counties_Eng_Wales.geojson', encoding='utf-8-sig') as f:
    data = json.load(f)
        

     # loop through the json file

    for feature in data['features']:
            
        # for each name in json file, ammend json file with numner of cases from dictionary
       
        if feature['properties']['ctyua16cd'] in covid_numbers.keys():
            feature['properties']['bng_e'] = covid_numbers[feature['properties']['ctyua16cd']]
        else:
            feature['properties']['bng_e']  = 0





# generate base map

england_map = folium.Map(location=[53.769790, -0.3], zoom_start=7,tiles = "Stamen Terrain")


fgcases = folium.FeatureGroup(name='Total Cases upto 21st June 2020')


# loop throught the geojson file

for feature in data['features']:
        
        lat = feature['properties']['lat']
        long = feature['properties']['long']
        cases = int(feature['properties']['bng_e'])
        name = feature['properties']['ctyua16nm']

        #determine traffic light code for area based on number of cases reported
        colour  = col_GOR(cases)
        
# add the counties Locations to the basemap as a point - both Circle and popup Marker
# pop menu to include name of authority and number of Covdi cases
        if(feature['properties']['ctyua16cd'][0] == 'E'):
            fgcases.add_child(folium.CircleMarker(location=[lat,long], 
              popup= None, radius = 7, fill_color = 'blue', 
              color = 'grey', fill_opacity = 1))
    
            fgcases.add_child(folium.Marker(location=[lat,long], 
                popup=name + '\n' + str(cases) + ' cases ', 
                icon=folium.Icon(color=colour)))
        

# read in Jason file for counties of England and Wales

fgareas = folium.FeatureGroup(name='Counties & Unified Authorities')

fgareas.add_child(folium.features.GeoJson(data , 
#style_function=lambda x: {'fillColor': 'grey' , 'fillOpacity' : 1}) )
#style_function=lambda x: {'fillColor': col_grey(x['properties']['bng_e']) , 'fillOpacity' : 1  }   ) )

#style_function=lambda x: {'fillColor': '#e8e8e8', 'fillOpacity' : 1  }   ) )

style_function=lambda x: {'fillColor': col_grey(x['properties']['bng_e']) , 
                          'fillOpacity' : 1  ,  'color': 'black' }
            # check that polygon is in England ('E')
            if x['properties']['ctyua16cd'][0] is 'E'  
            # else polygon is in Wales - make opacity =0 
                  else {'fillColor': 'red' ,'fillOpacity' : 0, 'color': 0} ))



# add child data and LayerControl functionality to the map & save

england_map.add_child(fgareas)
england_map.add_child(fgcases)
england_map.add_child(folium.LayerControl())



england_map.save("Map_England_Covid19.html")


The following code enables the output html file to be seen here in jupyter notebooks

In [1]:
from IPython.display import IFrame

IFrame(src='Map_England_Covid19.html', width=500, height=400)

Interestingly, the highest cases of postive tests are in the NW of England, the West Midlands and Kent. Parts of London (e.g. Richmond, Kingston) had very low number of cases by comparison.

The image belows is a screen capture of the output webpage:

<img src="output.png">